In [1]:
import os
from PIL import Image
import numpy as np
import ee
import geemap

In [2]:
#!python -m pip install geemap
#!export PATH="/home/u9999ad7367849478acc6568ce06912b/.local/bin:$PATH"
#!python -m pip install dask[complete]
#!pip install earthengine-api keras tensorflow
#!pip install geemap
#!pip install --user dask[dataframe]

In [3]:
#app = Flask(__name__)
ee.Authenticate()

True

In [4]:
def RetrieveNAIPimage(lat, long, date2, deltalr=0.00165, deltaud=0.00135, date1='2018-01-01'):
    region = ee.Geometry.BBox(long - deltalr, lat - deltaud, long + deltalr, lat + deltaud) 
    dataset = ee.ImageCollection('USDA/NAIP/DOQQ').filter(ee.Filter.date(date1, date2)).filter(ee.Filter.bounds(region))
    return dataset

def RetrieveNAIPimage2(lat, long, date2, deltalr=0.00165, deltaud=0.00135, date1='2015-01-01'):
    region = ee.Geometry.BBox(long - deltalr, lat - deltaud, long + deltalr, lat + deltaud) 
    dataset = ee.ImageCollection('USDA/NAIP/DOQQ').filter(ee.Filter.date(date1, date2)).filter(ee.Filter.bounds(region))
    return dataset

def DisplayMap(dataset, lat, long, verbose = True, deltalr = 0.00165, deltaud = 0.00135, zoom = 18):  
    image = dataset.first()
    trueColor = dataset.select(['R', 'G', 'B'])
    trueColorVis = {
      min: 50,
      max: 240,
    }
    if verbose:
        display("images found ",dataset.size())
        print(geemap.image_props(image).getInfo())
        print(geemap.image_stats(image, scale=18).getInfo())
        display(image.geometry().getInfo())
    region = ee.Geometry.BBox(long- deltalr, lat - deltaud, long + deltalr, lat + deltaud)
    regionDebug = ee.Geometry.Polygon(image.getInfo()['properties']['system:footprint']['coordinates'])
    Map = geemap.Map(center=(lat, long), zoom=18)
    Map.addLayer(trueColor, trueColorVis, 'True Color');
    Map.addLayer(region, {'color': 'red'}, '500x500 Image Selected')
    Map.addLayer(regionDebug, {'color': 'green'}, 'NAIP Image Boundary')
    #Map.setOptions('SATELLITE');
    image = dataset.first()

    return Map 

In [5]:
#def getScores(latitude, longitude)
ee.Initialize()
def generate_images(longitude,latitude):
    long, lat = longitude,latitude # DNM
    #lat, long = latitude, longitude
    date1 = '2024-02-08'
    dataset = RetrieveNAIPimage(lat, long, date1)
    date2 = '2017-02-08'
    dataset2 = RetrieveNAIPimage2(lat, long, date2)
    #Map = DisplayMap(dataset, lat, long, verbose = True, zoom = 18)
    #Map
    image1 = dataset.first()
    image2 = dataset2.first()
    deltalr = 0.00164
    deltaud = 0.00134
    region = ee.Geometry.BBox(long - deltalr, lat - deltaud, long + deltalr, lat + deltaud)
    rgb_img = geemap.ee_to_numpy(image1, bands=['R', 'G', 'B'], region=region)
    rgb_img
    rgb_img2 = geemap.ee_to_numpy(image2, bands=['R', 'G', 'B'], region=region)
    rgb_img2
    # Convert the RGB image to a PIL Image
    pil_img = Image.fromarray(np.uint8(rgb_img))
    pil_img2 = Image.fromarray(np.uint8(rgb_img2))
    save_image(pil_img,pil_img2)

def save_image(pil_img,pil_img2):
    # Save the PIL Image as JPEG
    output_filename = "output_image.jpg"
    pil_img.save(output_filename)
    # Save the PIL Image as JPEG
    output_filename = "output_image2.jpg"
    pil_img2.save(output_filename)

    #print("Image saved as", output_filename)

#rgb_img = ee.Image(image1).clipToBoundsAndScale(region=region).toInt().getInfo()
#rgb_img2 = ee.Image(image2).clipToBoundsAndScale(region=region).toInt().getInfo()


In [6]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image

# Load a pre-trained ResNet model
model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(1024, 1024))
    img_array = image.img_to_array(img)
    img_array = preprocess_input(img_array)
    img_array = tf.expand_dims(img_array, 0)  # Add batch dimension
    return img_array

def extract_features(image_tensor):
    features = model.predict(image_tensor)
    return features

def compare_images(image_path1, image_path2):
    image_tensor1 = preprocess_image(image_path1)
    image_tensor2 = preprocess_image(image_path2)
    
    features1 = extract_features(image_tensor1)
    features2 = extract_features(image_tensor2)
    
    # Calculate the cosine similarity between the two feature vectors
    cosine_similarity = tf.keras.losses.cosine_similarity(features1, features2)
    return cosine_similarity.numpy()[0]  # Extracting the scalar value from the tensor

def score_estate(longitude, latitude):
    generate_images(longitude, latitude)
    image_path1 = 'output_image.jpg'
    image_path2 = 'output_image2.jpg'

    similarity = compare_images(image_path1, image_path2)
    #print(f"Similarity between the images: {similarity}")

    score = similarity
    #(-3 ** (12 * (similarity - 1)) + 1) * 100
    return score

2024-02-11 05:10:48.389062: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-11 05:10:48.479289: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-11 05:10:48.479370: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-11 05:10:48.482205: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-11 05:10:48.497740: I tensorflow/core/platform/cpu_feature_guar

In [7]:
import googlemaps
def find_nearby_cities(api_key, location, radius=500000):
    # Initialize the Google Maps client
    #print(api_key)
    gmaps = googlemaps.Client(key=api_key)
    
    
    # Geocoding an address to get latitude and longitude
    geocode_result = gmaps.geocode(location)
    if not geocode_result:
        print(f"Could not find location: {location}")
        return []
    
    # Extract latitude and longitude
    lat = geocode_result[0]["geometry"]["location"]["lat"]
    lng = geocode_result[0]["geometry"]["location"]["lng"]
    
    # Use the Places API to find nearby places. Adjust the type as needed.
    # Here 'locality' is used to find cities/towns. You might need to adjust the type or use the 'keyword' parameter for different searches.
    places_result = gmaps.places_nearby(location=(lat, lng), radius=radius, type='locality')
    
    # Collect the names of nearby cities
    nearby_cities = []
    for place in places_result['results']:
        nearby_cities.append(place['name'])
    
    # Check if there are more results available
    while 'next_page_token' in places_result:
        # Fetch the next page of results using the next_page_token
        next_page_token = places_result['next_page_token']
        places_result = gmaps.places_nearby(location=(lat, lng), radius=radius, type='locality', page_token=next_page_token)
        
        # Collect the names of nearby cities
        for place in places_result['results']:
            nearby_cities.append(place['name'])
    
    return nearby_cities

# Example usage
api_key = 'PLEASE_INSERT_YOUR_GOOGLE_MAPS_API_KEY'


In [8]:
import requests
def get_lat_long(city, state):
    # Format city and state into a single query string
    query = f"{city}, {state}"

    # Base URL for the Nominatim API
    base_url = "https://nominatim.openstreetmap.org/search"

    # Parameters for the API request
    params = {
        "q": query,
        "format": "json",
    }

    # Send GET request to the API
    response = requests.get(base_url, params=params)
    data = response.json()

    # Extract latitude and longitude from the response
    if data:
        lat = data[0]['lat']
        lon = data[0]['lon']
        return float(lat), float(lon)
    else:
        print("Location not found.")
        return None, None

def recommendation():
    # Prompt the user for city and state
    desirable_city = input("Enter the city name: ")
    state = input("Enter the state name: ")
    
    neighbour = find_nearby_cities(api_key,desirable_city)
    
    if desirable_city in neighbour:
        neighbour.remove(desirable_city)
    
    lat_cur, long_cur = get_lat_long(desirable_city, state)
    
    lat_nb, long_nb = get_lat_long(neighbour[0], state)
    
    #print(neighbour[0])
    
    score_cur = score_estate(long_cur, lat_cur)
    score_neighbour = score_estate(long_nb, lat_nb)
    
    print("The recommendation score of " + desirable_city + " is " + str(score(score_cur)))
    print("The recommendation score of " + neighbour[0] + " is " + str(score(score_neighbour)))

    
    #print(score(score_neighbour))

def score(val):
    #return 100*(-15**val + 1)
    return (1+(val/2))*100

recommendation()


Enter the city name:  Atlanta
Enter the state name:  Georgia


1/1 [==============================] - 0s 321ms/step
The recommendation score of Atlanta is 71.80052399635315
The recommendation score of Marietta is 62.57222890853882
